原文代码作者：https://github.com/wzyonggege/statistical-learning-method,  https://www.cnblogs.com/21207-iHome/p/6084670.html

中文注释制作：机器学习初学者(微信公众号：ID:ai-start-com)

配置环境：python 3.6

代码全部测试通过。
![gongzhong](../gongzhong.jpg)

# 第4章 朴素贝叶斯

基于贝叶斯定理与特征条件独立假设的分类方法。

模型：

- 高斯模型
- 多项式模型
- 伯努利模型

In [165]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math

In [166]:
# data
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, :])
    # print(data)
    return data[:,:-1], data[:,-1]

In [167]:
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [168]:
X_test[0], y_test[0]

(array([5.4, 3.9, 1.3, 0.4]), 0.0)

参考：https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/

## GaussianNB 高斯朴素贝叶斯

特征的可能性被假设为高斯

概率密度函数：
$$P(x_i | y_k)=\frac{1}{\sqrt{2\pi\sigma^2_{yk}}}exp(-\frac{(x_i-\mu_{yk})^2}{2\sigma^2_{yk}})$$

数学期望(mean)：$\mu$，方差：$\sigma^2=\frac{\sum(X-\mu)^2}{N}$

In [169]:
class NaiveBayes:
    def __init__(self):
        self.model = None

    # 数学期望 就是平均值
    @staticmethod
    def mean(X):
        return sum(X) / float(len(X))

    # 标准差（方差）
    def stdev(self, X):
        avg = self.mean(X)
        return math.sqrt(sum([pow(x-avg, 2) for x in X]) / float(len(X)))

    # 概率密度函数
    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
        return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

    # 处理X_train
    def summarize(self, train_data):
        #print("train_data",train_data)
        summaries = [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]
        #print("summaries",summaries)
        return summaries

    # 分类别求出数学期望和标准差
    def fit(self, X, y):
        labels = list(set(y))
        print('labels',labels)
        data = {label:[] for label in labels}
        print("data",data)
        for f, label in zip(X, y):
            data[label].append(f)
            #print('data',data)
        self.model = {label: self.summarize(value) for label, value in data.items()}
        print('self.model',self.model)
        return 'gaussianNB train done!'

    # 计算概率
    def calculate_probabilities(self, input_data):
        # summaries:{0.0: [(5.0, 0.37),(3.42, 0.40)], 1.0: [(5.8, 0.449),(2.7, 0.27)]}
        # input_data:[1.1, 2.2]
        probabilities = {}
        for label, value in self.model.items():
            probabilities[label] = 1
            for i in range(len(value)):
                mean, stdev = value[i]
                probabilities[label] *= self.gaussian_probability(input_data[i], mean, stdev)
        print("probabilities",probabilities)
        return probabilities

    # 类别
    def predict(self, X_test):
        # {0.0: 2.9680340789325763e-27, 1.0: 3.5749783019849535e-26}
        
        #print("pre",sorted(self.calculate_probabilities(X_test).items(), key=lambda x: x[-1]))
        #print("pre1",sorted(self.calculate_probabilities(X_test).items(), key=lambda x: x[-1])[-1])
        #print("pre2",sorted(self.calculate_probabilities(X_test).items(), key=lambda x: x[-1])[-1][0])
        
        label = sorted(self.calculate_probabilities(X_test).items(), key=lambda x: x[-1])[-1][0]
        return label

    def score(self, X_test, y_test):
        right = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right += 1

        return right / float(len(X_test))

In [170]:
model = NaiveBayes()

In [171]:
model.fit(X_train, y_train)

labels [0.0, 1.0]
data {0.0: [], 1.0: []}
self.model {0.0: [(5.015625, 0.3666651870235297), (3.4687500000000004, 0.34860211918460854), (1.4625, 0.18498310733685927), (0.259375, 0.1114096466873493)], 1.0: [(5.973684210526316, 0.4749981775734102), (2.7736842105263158, 0.2953002134548536), (4.26842105263158, 0.46741464920734865), (1.3210526315789475, 0.19215932908977562)]}


'gaussianNB train done!'

In [172]:
print(model.predict([4.4,  3.2,  1.3,  0.2]))

probabilities {0.0: 1.0293054484089323, 1.0: 2.0831593056719582e-19}
pre [(1.0, 2.0831593056719582e-19), (0.0, 1.0293054484089323)]
probabilities {0.0: 1.0293054484089323, 1.0: 2.0831593056719582e-19}
pre1 (0.0, 1.0293054484089323)
probabilities {0.0: 1.0293054484089323, 1.0: 2.0831593056719582e-19}
pre2 0.0
probabilities {0.0: 1.0293054484089323, 1.0: 2.0831593056719582e-19}
0.0


In [173]:
model.score(X_test, y_test)

probabilities {0.0: 0.7915932954933907, 1.0: 1.2045563337518497e-17}
pre [(1.0, 1.2045563337518497e-17), (0.0, 0.7915932954933907)]
probabilities {0.0: 0.7915932954933907, 1.0: 1.2045563337518497e-17}
pre1 (0.0, 0.7915932954933907)
probabilities {0.0: 0.7915932954933907, 1.0: 1.2045563337518497e-17}
pre2 0.0
probabilities {0.0: 0.7915932954933907, 1.0: 1.2045563337518497e-17}
probabilities {0.0: 0.4198867618328872, 1.0: 1.0833417207250703e-15}
pre [(1.0, 1.0833417207250703e-15), (0.0, 0.4198867618328872)]
probabilities {0.0: 0.4198867618328872, 1.0: 1.0833417207250703e-15}
pre1 (0.0, 0.4198867618328872)
probabilities {0.0: 0.4198867618328872, 1.0: 1.0833417207250703e-15}
pre2 0.0
probabilities {0.0: 0.4198867618328872, 1.0: 1.0833417207250703e-15}
probabilities {0.0: 1.656507416560436e-95, 1.0: 0.04122997807964096}
pre [(0.0, 1.656507416560436e-95), (1.0, 0.04122997807964096)]
probabilities {0.0: 1.656507416560436e-95, 1.0: 0.04122997807964096}
pre1 (1.0, 0.04122997807964096)
probabili

1.0

scikit-learn实例

# sklearn.naive_bayes

In [174]:
from sklearn.naive_bayes import GaussianNB

In [175]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [176]:
clf.score(X_test, y_test)

1.0

In [177]:
clf.predict([[4.4,  3.2,  1.3,  0.2]])

array([0.])

In [178]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB # 伯努利模型和多项式模型